<h2>Data Wrangling</h2>

The goal of this step is mainly to convert the source Leica file into individual 3D tiff files. For some data, the current python reader cannot load the channel dimenstion and the Z dimension correctly. So, we have to re-order the data. We only need such dimension fix on the very first dataset. 

Parameter selection

<b>Use dimension fix</b>:<br>
    For future data, just try to run without dimension fix. 
    If the output is not correct, then re-run with dimension fix.

<b>Channel specification</b>:<br>
        We asume CD31 Channel as 1 and Col IV Channel as 2 (0 based index), if it's not the case in your images, please provide your own channel references:<br><br>
        <b>CD31 Channel</b>: Platelet endothelial cell adhesion marker channel<br>
        <b>Col IV Channel</b>: Extracellular matrix marker channel<br><br>
        
<b>LIF files path</b>:<br>
        you file path to data /path/to/string_vessel_data/
    
<b>Output path</b>:<br>
        specific where you want to save the splitted files
        /path/to/output_folder/
        
    
    
    
        
   
    

In [ ]:
from core.image_processor import run_image_processing_menu
run_image_processing_menu()